In [1]:
%matplotlib inline

import jinja2
import json
import requests
import StringIO

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from collections import OrderedDict
from datetime import datetime
from dateutil import parser
from io import BytesIO
from IPython.html.widgets import interact
from IPython.html import widgets
from IPython.display import display, display_pretty, Javascript, HTML
from IPython.utils.traitlets import Unicode
from mpl_toolkits.basemap import Basemap
from xml.etree import ElementTree

:0: FutureWarning: IPython widgets are experimental and may change in the future.


In [4]:
def display_heat_points_map_ol(latitude, longitude, div):
    # Make the request and parse the response as XML using ElementTree
    response = requests.get('http://catalogos.conae.gov.ar/focos/focosdecalor.xml')
    response_atom = ElementTree.parse(BytesIO(response.content))

    # Build a DataFrame extrating the data for each entry using XPath
    entries_id = [e.text for e in response_atom.findall('.//{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}id')]
    entries_title = [e.text for e in response_atom.findall('.//{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}title')]
    entries_updated = [e.text for e in response_atom.findall('.//{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}updated')]
    entries_summary = [e.text for e in response_atom.findall('.//{http://www.w3.org/2005/Atom}entry/{http://www.w3.org/2005/Atom}summary')]
    entries_point_lat = [float(e.text.split(" ")[0]) for e in response_atom.findall('.//{http://www.w3.org/2005/Atom}entry/{http://www.georss.org/georss}point')]
    entries_point_lon = [float(e.text.split(" ")[1]) for e in response_atom.findall('.//{http://www.w3.org/2005/Atom}entry/{http://www.georss.org/georss}point')]

    entries_df = pd.DataFrame({
        'id': entries_id,
        'title': entries_title,
        'summary': entries_summary,
        'lat': entries_point_lat,
        'lon': entries_point_lon
    }, index=entries_updated)

    # Define a Jinja2 template that creates the map, and iterates of the
    # entries to create a marker with a custom icon for each of them.
    heat_points_javascript = jinja2.Template(
    """
    require(["open_layers"], function() {
        var map = new OpenLayers.Map('heat_points_ol', {});
        // var layer = new OpenLayers.Layer.WMS('OpenLayers WMS', 'http://vmap0.tiles.osgeo.org/wms/vmap0', {layers: 'basic'});
        
        var layer = new OpenLayers.Layer.WMS(
                'Global Imagery',
                'http://maps.opengeo.org/geowebcache/service/wms',
                {layers: 'bluemarble'},
                {isBaseLayer: true}
        );
        
        map.addLayer(layer);
        map.setCenter(
            new OpenLayers.LonLat({{ longitude }}, {{ latitude }})
        );
        
        var heat_points_layer = new OpenLayers.Layer.Markers("Heat Points Layer");
        
        var machinalis_logo_icon_url = 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAB4AAAAeCAYAAAA7MK6iAAAABmJLR0QA/wD/AP+gvaeTAAAACXBIWXMAAAsTAAALEwEAmpwYAAAAB3RJTUUH3gUdESIC2is5hgAAAB1pVFh0Q29tbWVudAAAAAAAQ3JlYXRlZCB3aXRoIEdJTVBkLmUHAAAF9klEQVRIx72WaVBTVxTH/29LYgCxskQkLlSgSFSMYqmgVRSjIoxUpVrHqrWiWMcFl7rUDrU6Mu1UaV1ah6mOSGHccBxrnSq4VS1QQcRQjIIgiAUEFRASs7x3+yESQxIRmE7vpzfv3nt+95zzP+deoBujMHbDyjyXcJLLhRF1WFxVUKCC6aoNujtg44073xG9AaAptObf6bdBOlj5v4BB220j/zn4aMTCThkqW7TN8p2lWvLG9eybFszOPoSLk+PneurI96VyacAHGclPHa3zPfglyhZtO1L74KHrmPMpUzvt8TpXhd3keP8hLjcDPix2unYnvSWn2IPTGvp0ZExbVj2BuaKeku8dpS349KulnQJv3vJFv0ynkHX3P/t2FACkKqcHJlOBzfqyagUx8QBNARTVYS4pmjZSNA1TQ2MPY+r5/fkRy1YDwG9LtsjOeIQvvTAsdjIAJPmHvwq1Zmfqr95GLqj+wGkURq46T0qrw/VV/4Biuqc/MDRMl24lF83ZFG04lR/ONxkpmmpGkn84t+neJRMNAJkTFkYxrYYgUOY9+qx8laGylrMYEQjY3j1hchY3dsTiPHrVEkGwUhADXeaVCXxjCwUKEJ49h8pzkNoSapahtbSIA9o2mcNqgdKyt1pefDJpcEzazprXQZtX7UHA8aSRxM/7D7vSo8wFR7EMOIm4wQKennXwYtFQd4VkuH+Tnex794Ru4UTlmB0Jmo687fnDCgBAqObEOOfRQ0vsIu/qBFHs+MRh2fvGthPX4ssZJS5BfhG2GwR/73XjtieUdTa1pQu24oVvn2lEaK/DZi/XA8rD27522ECqLuQEt1Mpx0IX+s4vu5VRndaUX2oign9OfCCdFFwIq3w7CdT8dpG8MWNtnFBcsRsA6GYdx1sJShoSWBXyzfo6O+vWAnp5Rtsf1yrvJo2SiI8JBqN5QUUtdyNglk4QBFByz2ssybqZQvQGAAAPysoEgUjued1hvfaXPUJV/UBQAOvmihYJ1WCXUxfpaUpCgJdgYuLBl9dIAIBUPo5ghRfmW8ZxfbB25VM9dyvkGYk+l2clLAdPJOWc4eySYz8+sFN5D1ZvZ7etUggBK1X612mLSmWOuPyz58Nt/8kzEs3t9ETyvg5V3qwfBiPv+ILw6GVi97o09JuslI8w8bxxQCvicL8mvu102rySYHRzKOQDlvHqAouXEp++KB/QU0GaWpy1Ura2XSzygj9WEXXFuVe3LAGJUE4cfXbvxc4CT0XFg14zQySP2681PKxj2sCtPp45EzUnQx2WE9PXbYptTqTPdOld8TTmzH64f354gaH6sQUKAJ4eHrLEoRNk7cApo2d6F/jOqDedy0+wNaTLv9MnJ2jOGQC4t3j7a4Gno8wZuh67fjZXXpdiO9+So347qoarLfloc5IFLGOlK0xVde7WIbbutfTdR9PU4cuuNfagvayN7QpUWb41q6ewuaMXxosu3DrCt+pga4tiaPDNLTDcrlhvpW/gr76RrcKTZikAcHIPo+FhPUfZVgPLQqwKzr6iKdrxgqOvbrh9zrQrUDUoOEAR51JcvUZfWcO1bzQEIm8P3lDTwJjhDO4rvebPvX4k7RVYtXy+UNNwsMmg31gTM+KnIXnVCaar6m1gGbuuRTv1AC0RARQFYuTBP281e2Z9UkJg7OdedjXSb+TMevHwJ+p76bSztOi9vNSojLA5cNg5dgZEYK0mGwWRK4uNWfkKB6/KjgcBmF7OpHJeaB9xYXl99KVD5I09tm2cmbt2oOfZWxWCTt/dUoYwZMDs0IL0Y1163nrerVtkgRICTtYbpsD+JxgfL1A2yRfLZTD6epVQwwf9bi1MsdZ4tMvv6kadVsO4OJlvIoaBJDp0/pjCjNhRmuMU3dfNaA24O8Tdf+zfxxUhealTaTfXVggCKJoGLxXf7DJYVXIyI7e/yJULDngkCVVkK/ZtTHulm/YpM9U+dbb06JixCU4jAwzVo/qvebcgbWSX87NnRPRr5wrGL/0zVzKW5IrCSMHAGENKWKzc0brM9+e91sa/SYtOrUfMruYAAAAASUVORK5CYII=';
        var heat_point_icon_url = 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABUAAAAgCAYAAAD9oDOIAAAABmJLR0QA/wD/AP+gvaeTAAAACXBIWXMAAA3XAAAN1wFCKJt4AAAAB3RJTUUH3gUcDQEVStH2UQAAArJJREFUSMet1UtoXGUUB/Dfdyc0j0JREmybSBaKL6ILxTRYXUQQFAQXlUIXpQWzUFS6Koi7SnFpNz6ouml9RKpI6SYUurGCzcKtglVQWm2h1Edt7CN1JsfF/XKbiZnMJOmB4c537rl//uf1/1ihxc1nEYwHA4tjipWCphKwH3/jUuL3sEYL9gUR7HcLwLqCqQwYwV2xBrD53+kgoihBbwXLL4OIFyqWPy8XX3TQ5f3Y5gAuVa/n1sKwN6jHdhHfiHikYlpfFdNsJwyqeT1TvwlVCwZiFemP4HFv4Gp29jaFjKVVMN3lXjy0wDPS9P7d5RZE0JO4vojpFUf0uXNB1/7Ec03fv5R4vxXT2eCzoJYB73aHPkOaJ3Ijnm/6fh//H9oi0w1cwLns32Jwibz+xYvorjybgo/TMjX9ERuDNzFguEXB1uMoeirPzmC0Fej9+fkaety2zFZswCdN3qNNoAvqsXV+BrGhGqNWNohD1WkoZ1iCZn3sxsO2VUGXnWkDOof7cKDyvBp0LUz/U+sVdlcBF5zvYI8bGMM4uSgHIUWZyDm78Qqexl+mccYxOwxoL3QNPFnJTH+RjzyaR+Y98BhO2ysn1MbW4eXqNF5gO7g9MxrGBNjjJx+ant+7Nky3VqcdKTiPzb7A5uzuw6RyAWfMOqZbfxvgelXb31JwDT0+z2OymMHX+BW72oB24wnManRlmRg0s0RgDU/l9Osd1LZs1MUicymXtGhRr04AL+dGM13gMLnrfWu4Hc9W/yYLnMB3ZvJ2pFUA9uIIyv5MFWikUt/P+iiL30qBv8VxgZHE1cUqNWUnfuhw6FOu495y6BO/xLygZLueeNYNEyZ87x1czKNSW0KF1+EUnnHKP+5JueGpzZ0/HHwQo/6It9yIk+px0lx8pRFvm40HHA8eXOo6+Q9fYtvU17HpOgAAAABJRU5ErkJggg==';

        var machinalis_logo_icon = new OpenLayers.Icon(machinalis_logo_icon_url, new OpenLayers.Size(30, 30));
        heat_points_layer.addMarker(new OpenLayers.Marker(new OpenLayers.LonLat(-64.2010031, -31.4122599), machinalis_logo_icon));

        var heat_point_icon = new OpenLayers.Icon(heat_point_icon_url, new OpenLayers.Size(21, 32));
        
        {% for heat_point in heat_points %}
        heat_points_layer.addMarker(new OpenLayers.Marker(new OpenLayers.LonLat({{ heat_point[3] }}, {{ heat_point[2] }}), heat_point_icon.clone()));
        {% endfor %}
        
        map.addLayer(heat_points_layer);
    })
    """
    )
    # Render the template and display it as Javascript
    display(Javascript(heat_points_javascript.render(
        heat_points=entries_df.itertuples(),
        latitude=latitude, longitude=longitude
    )))

In [5]:
i = interact(
    display_heat_points_map_ol,
    latitude=widgets.FloatText(value=-31.4122599, min=-90.0, max=90.0),
    longitude=widgets.FloatText(value=-64.2010031, min=-180.0, max=180.0),
    div=widgets.HTML(value='<div id="heat_points_ol" style="width: 800px; height: 600px"></div>'),
)

<IPython.core.display.Javascript object>